# 🎯 Propensity Score Matching: Starter Template
## Hands-On Practice Notebook

**INFO 7390: Advanced Data Science and Architecture**  
**Author:** Nikshipth Narayan Bondugula  
**Date:** December 2025

---

## 🎯 Your Mission

In this notebook, you will implement a **Propensity Score Matching (PSM)** pipeline from scratch to estimate the causal effect of a cardiac rehabilitation program on hospital readmission.

**You will:**
1. Explore the data and identify selection bias
2. Estimate propensity scores using logistic regression
3. Perform nearest neighbor matching
4. Assess covariate balance
5. Estimate the Average Treatment Effect on the Treated (ATT)

---

## 📋 Instructions

- Look for `# TODO:` comments — these are sections YOU need to complete
- Hints are provided to guide you
- Expected outputs are shown so you can verify your work
- If you get stuck, refer to the complete tutorial notebook

---

## Difficulty Level: ⭐⭐⭐ Intermediate

**Estimated Time:** 45-60 minutes

---

## Step 0: Setup and Imports

Run this cell to import all required libraries. **No changes needed here.**

In [1]:
# =============================================================================
# SETUP: Import Required Libraries (No changes needed)
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 1: Load and Explore the Data

We'll use a synthetic dataset about a **cardiac rehabilitation program**.

**Research Question:** Does participating in cardiac rehab reduce 30-day hospital readmission?

**The Challenge:** Patients self-select into the program, creating selection bias.

### Variables in the Dataset:

| Variable | Description | Role |
|----------|-------------|------|
| `age` | Patient age (45-85) | Confounder |
| `severity` | Disease severity (1-10) | Confounder |
| `prior_admits` | Previous hospitalizations (0-5) | Confounder |
| `insurance` | Has premium insurance (0/1) | Confounder |
| `motivation` | Health motivation score (1-10) | Confounder |
| `treatment` | Enrolled in cardiac rehab (0/1) | Treatment |
| `readmitted` | Readmitted within 30 days (0/1) | Outcome |

In [2]:
# =============================================================================
# STEP 1A: Data Generation Function (No changes needed - just run this)
# =============================================================================

def generate_cardiac_rehab_data(n_patients=2000, true_treatment_effect=-0.15, seed=42):
    """
    Generate synthetic cardiac rehabilitation dataset with realistic confounding.
    """
    np.random.seed(seed)
    
    # Generate confounders
    age = np.random.normal(65, 10, n_patients).clip(45, 85)
    severity = np.random.normal(5, 2, n_patients).clip(1, 10)
    prior_admits = np.random.poisson(1.5, n_patients).clip(0, 5)
    insurance = np.random.binomial(1, 0.4, n_patients)
    motivation = np.random.normal(5, 2, n_patients).clip(1, 10)
    
    # Generate treatment with selection bias
    treatment_logit = (
        -2.0
        - 0.03 * (age - 65)
        - 0.2 * (severity - 5)
        - 0.3 * prior_admits
        + 0.8 * insurance
        + 0.3 * (motivation - 5)
    )
    true_propensity = 1 / (1 + np.exp(-treatment_logit))
    treatment = np.random.binomial(1, true_propensity)
    
    # Generate outcomes
    baseline_logit = (
        -1.5
        + 0.02 * (age - 65)
        + 0.25 * (severity - 5)
        + 0.3 * prior_admits
        - 0.15 * insurance
        - 0.15 * (motivation - 5)
    )
    
    prob_y0 = 1 / (1 + np.exp(-baseline_logit))
    prob_y1 = 1 / (1 + np.exp(-(baseline_logit + true_treatment_effect * 5)))
    
    y0 = np.random.binomial(1, prob_y0)
    y1 = np.random.binomial(1, prob_y1)
    readmitted = np.where(treatment == 1, y1, y0)
    
    df = pd.DataFrame({
        'patient_id': range(1, n_patients + 1),
        'age': np.round(age, 1),
        'severity': np.round(severity, 2),
        'prior_admits': prior_admits,
        'insurance': insurance,
        'motivation': np.round(motivation, 2),
        'treatment': treatment,
        'readmitted': readmitted,
        '_true_propensity': true_propensity,
        '_y0': y0,
        '_y1': y1
    })
    
    return df

# Generate the dataset
df = generate_cardiac_rehab_data(n_patients=2000)

print("✅ Dataset generated!")
print(f"   Shape: {df.shape[0]} patients, {df.shape[1]} variables")

✅ Dataset generated!
   Shape: 2000 patients, 11 variables


### TODO 1: Explore the Data

Complete the code below to explore the dataset and understand the selection bias problem.

In [ ]:
# =============================================================================
# TODO 1: Explore the Data
# =============================================================================

# TODO 1.1: Display the first 5 rows of the dataset
# Hint: Use df.head()

# YOUR CODE HERE:


# TODO 1.2: Calculate the number of treated and control patients
# Hint: Use df['treatment'].sum() for treated count

# YOUR CODE HERE:
n_treated = ____
n_control = ____

print(f"Treated patients: {n_treated}")
print(f"Control patients: {n_control}")


# TODO 1.3: Calculate readmission rates for each group
# Hint: Use df[df['treatment'] == 1]['readmitted'].mean() for treated group

# YOUR CODE HERE:
readmit_treated = ____
readmit_control = ____

print(f"Readmission rate (Treated): {readmit_treated*100:.1f}%")
print(f"Readmission rate (Control): {readmit_control*100:.1f}%")
print(f"Naive difference: {(readmit_treated - readmit_control)*100:.1f} percentage points")

#### ✅ Expected Output (approximately):
```
Treated patients: ~600-700
Control patients: ~1300-1400
Readmission rate (Treated): ~15-20%
Readmission rate (Control): ~22-28%
Naive difference: ~-5 to -10 percentage points
```

**❓ Question:** Why is the naive difference NOT the true causal effect?

*Your answer:* _______________

---

## TODO 2 Estimate Propensity Scores

Now you'll build a logistic regression model to estimate each patient's probability of receiving treatment.

**Covariates to use:** `age`, `severity`, `prior_admits`, `insurance`, `motivation`

In [ ]:
# =============================================================================
# TODO 2: Estimate Propensity Scores
# =============================================================================

# Define covariates (provided for you)
covariates = ['age', 'severity', 'prior_admits', 'insurance', 'motivation']

# TODO 2.1: Prepare the features (X) and target (y)
# Hint: X = df[covariates], y = df['treatment']

# YOUR CODE HERE:
X = ____
y = ____


# TODO 2.2: Standardize the features using StandardScaler
# Hint: Create a StandardScaler, then use fit_transform(X)

# YOUR CODE HERE:
scaler = ____
X_scaled = ____


# TODO 2.3: Fit a logistic regression model
# Hint: Create LogisticRegression(max_iter=1000), then use .fit(X_scaled, y)

# YOUR CODE HERE:
model = ____
____  # fit the model


# TODO 2.4: Get propensity scores (predicted probabilities)
# Hint: Use model.predict_proba(X_scaled)[:, 1]

# YOUR CODE HERE:
propensity_scores = ____


# Add propensity scores to dataframe
df['propensity_score'] = propensity_scores

print("✅ Propensity scores estimated!")
print(f"   Min: {propensity_scores.min():.4f}")
print(f"   Max: {propensity_scores.max():.4f}")
print(f"   Mean: {propensity_scores.mean():.4f}")

#### ✅ Expected Output (approximately):
```
✅ Propensity scores estimated!
   Min: ~0.05-0.10
   Max: ~0.85-0.95
   Mean: ~0.30-0.40
```

### Step 2: Visualize Propensity Score Overlap

Check whether there's sufficient overlap between treated and control groups.

In [ ]:
# =============================================================================
# TODO 3: Visualize Propensity Score Distributions
# =============================================================================

# TODO 3.1: Create a histogram showing PS distributions for both groups
# Hint: 
#   - Use plt.hist() for each group
#   - df[df['treatment'] == 0]['propensity_score'] gives control PS
#   - df[df['treatment'] == 1]['propensity_score'] gives treated PS
#   - Use alpha=0.6 for transparency, density=True for normalization

plt.figure(figsize=(10, 6))

# YOUR CODE HERE:
# Plot control group histogram


# Plot treated group histogram


plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('Propensity Score Distribution by Treatment Group')
plt.legend()
plt.show()


# TODO 3.2: Calculate the region of common support
# Hint: Common support is from max(min of both groups) to min(max of both groups)

# YOUR CODE HERE:
ps_control = df[df['treatment'] == 0]['propensity_score']
ps_treated = df[df['treatment'] == 1]['propensity_score']

common_support_min = ____  # max of the two minimums
common_support_max = ____  # min of the two maximums

print(f"Region of Common Support: [{common_support_min:.4f}, {common_support_max:.4f}]")

#### ✅ Expected Output:
- Histogram should show two overlapping distributions
- Common support region should span most of the PS range
- If distributions don't overlap, PSM may not be appropriate!

---

## Step 3: Perform Matching

Now implement nearest neighbor matching to find comparable pairs.

In [ ]:
# =============================================================================
# TODO 4: Implement Nearest Neighbor Matching
# =============================================================================

def nearest_neighbor_matching(df, propensity_col='propensity_score', 
                               treatment_col='treatment', caliper=None):
    """
    Perform nearest neighbor matching on propensity scores.
    
    Parameters:
    -----------
    df : DataFrame with propensity scores
    propensity_col : Name of propensity score column
    treatment_col : Name of treatment column
    caliper : Maximum allowed distance (None = no limit)
    
    Returns:
    --------
    matched_df : DataFrame with matched pairs
    match_info : Dictionary with matching statistics
    """
    
    # Separate treated and control
    treated = df[df[treatment_col] == 1].copy()
    control = df[df[treatment_col] == 0].copy()
    
    # Store matched pairs
    matched_treated_idx = []
    matched_control_idx = []
    match_distances = []
    
    # Track available controls
    available_control_idx = set(control.index.tolist())
    unmatched_count = 0
    
    # TODO 4.1: For each treated unit, find the nearest control
    # This loop is partially completed - fill in the missing parts
    
    for treated_idx in treated.index:
        # Get propensity score of this treated unit
        treated_ps = treated.loc[treated_idx, propensity_col]
        
        # Get available controls
        available_controls = control.loc[list(available_control_idx)]
        
        if len(available_controls) == 0:
            unmatched_count += 1
            continue
        
        # TODO: Calculate distance to all available controls
        # Hint: Use np.abs(available_controls[propensity_col] - treated_ps)
        
        # YOUR CODE HERE:
        distances = ____
        
        
        # TODO: Find the nearest neighbor (control with minimum distance)
        # Hint: Use distances.idxmin() for index, distances.min() for value
        
        # YOUR CODE HERE:
        nearest_idx = ____
        nearest_distance = ____
        
        
        # Check caliper (if specified)
        if caliper is not None and nearest_distance > caliper:
            unmatched_count += 1
            continue
        
        # Record the match
        matched_treated_idx.append(treated_idx)
        matched_control_idx.append(nearest_idx)
        match_distances.append(nearest_distance)
        
        # Remove control from available pool (matching without replacement)
        available_control_idx.discard(nearest_idx)
    
    # Create matched dataset
    matched_treated = df.loc[matched_treated_idx].copy()
    matched_treated['match_id'] = range(len(matched_treated_idx))
    
    matched_control = df.loc[matched_control_idx].copy()
    matched_control['match_id'] = range(len(matched_control_idx))
    
    matched_df = pd.concat([matched_treated, matched_control], axis=0)
    matched_df = matched_df.sort_values(['match_id', treatment_col], ascending=[True, False])
    
    # Compile statistics
    match_info = {
        'n_treated_original': len(treated),
        'n_matched_pairs': len(matched_treated_idx),
        'n_unmatched': unmatched_count,
        'mean_distance': np.mean(match_distances) if match_distances else None
    }
    
    return matched_df, match_info


# TODO 4.2: Calculate caliper (0.2 × standard deviation of propensity scores)
# YOUR CODE HERE:
caliper = ____

print(f"Caliper: {caliper:.4f}")


# TODO 4.3: Perform matching using the function above
# YOUR CODE HERE:
matched_df, match_info = ____


# Print results
print(f"\n✅ Matching completed!")
print(f"   Matched pairs: {match_info['n_matched_pairs']}")
print(f"   Unmatched treated: {match_info['n_unmatched']}")
print(f"   Mean match distance: {match_info['mean_distance']:.4f}")

#### ✅ Expected Output (approximately):
```
Caliper: ~0.03-0.05
✅ Matching completed!
   Matched pairs: ~550-650
   Unmatched treated: ~20-80
   Mean match distance: ~0.005-0.015
```

---

## Step 4: Assess Covariate Balance

Check whether matching successfully balanced the confounders.

In [ ]:
# =============================================================================
# TODO 5: Calculate Standardized Mean Differences (SMD)
# =============================================================================

def calculate_smd(treated_values, control_values):
    """
    Calculate Standardized Mean Difference.
    
    SMD = (mean_treated - mean_control) / pooled_std
    """
    mean_treated = treated_values.mean()
    mean_control = control_values.mean()
    
    # TODO 5.1: Calculate pooled standard deviation
    # Formula: sqrt((var_treated + var_control) / 2)
    
    # YOUR CODE HERE:
    var_treated = ____
    var_control = ____
    pooled_std = ____
    
    if pooled_std == 0:
        return 0.0
    
    smd = (mean_treated - mean_control) / pooled_std
    return smd


# TODO 5.2: Calculate SMD for each covariate before and after matching
print("=" * 60)
print("COVARIATE BALANCE: Before vs After Matching")
print("=" * 60)
print(f"\n{'Covariate':<15} {'SMD Before':<15} {'SMD After':<15} {'Status'}")
print("-" * 60)

for covar in covariates:
    # Before matching
    treated_before = df[df['treatment'] == 1][covar]
    control_before = df[df['treatment'] == 0][covar]
    smd_before = calculate_smd(treated_before, control_before)
    
    # TODO: Calculate SMD after matching
    # Hint: Use matched_df instead of df
    
    # YOUR CODE HERE:
    treated_after = ____
    control_after = ____
    smd_after = ____
    
    # Determine status
    if abs(smd_after) < 0.1:
        status = "✅ Excellent"
    elif abs(smd_after) < 0.25:
        status = "⚠️ Acceptable"
    else:
        status = "❌ Poor"
    
    print(f"{covar:<15} {smd_before:<15.4f} {smd_after:<15.4f} {status}")

#### ✅ Expected Output:
- All SMD After values should be close to 0 (ideally < 0.1)
- All covariates should show "✅ Excellent" or "⚠️ Acceptable"
- If any show "❌ Poor", matching may need adjustment

---

## Step 5: Estimate Treatment Effect (ATT)

Finally, estimate the Average Treatment Effect on the Treated!

In [ ]:
# =============================================================================
# TODO 6: Estimate the Average Treatment Effect on the Treated (ATT)
# =============================================================================

# TODO 6.1: Calculate mean outcomes for treated and control in matched sample

# YOUR CODE HERE:
mean_treated = ____  # Hint: matched_df[matched_df['treatment'] == 1]['readmitted'].mean()
mean_control = ____  # Hint: matched_df[matched_df['treatment'] == 0]['readmitted'].mean()


# TODO 6.2: Calculate ATT (difference in means)

# YOUR CODE HERE:
att = ____


# TODO 6.3: Calculate standard error and 95% confidence interval
n_pairs = match_info['n_matched_pairs']
var_treated = matched_df[matched_df['treatment'] == 1]['readmitted'].var()
var_control = matched_df[matched_df['treatment'] == 0]['readmitted'].var()

# YOUR CODE HERE:
se_att = ____  # Hint: np.sqrt((var_treated + var_control) / n_pairs)
ci_lower = ____  # Hint: att - 1.96 * se_att
ci_upper = ____  # Hint: att + 1.96 * se_att


# Print results
print("=" * 60)
print("AVERAGE TREATMENT EFFECT ON THE TREATED (ATT)")
print("=" * 60)
print(f"\nReadmission Rates:")
print(f"   Treated: {mean_treated*100:.2f}%")
print(f"   Control: {mean_control*100:.2f}%")
print(f"\n🎯 ATT Estimate: {att*100:.2f} percentage points")
print(f"   95% CI: [{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]")

# Compare to naive estimate
naive_att = df[df['treatment']==1]['readmitted'].mean() - df[df['treatment']==0]['readmitted'].mean()
print(f"\n📊 Comparison:")
print(f"   Naive estimate: {naive_att*100:.2f} pp")
print(f"   PSM estimate:   {att*100:.2f} pp")
print(f"   True effect:    ~-15 pp (built into data)")

#### ✅ Expected Output (approximately):
```
🎯 ATT Estimate: -10 to -15 percentage points
   95% CI: Should not include 0 if effect is significant

📊 Comparison:
   Naive estimate: ~-5 to -8 pp (biased toward 0)
   PSM estimate:   ~-10 to -15 pp (closer to truth!)
   True effect:    ~-15 pp
```

**🎉 Success!** Your PSM estimate should be much closer to the true effect than the naive estimate!

---

## 🎓 Summary: What You Accomplished

In this notebook, you:

1. ✅ Explored data and identified selection bias
2. ✅ Estimated propensity scores using logistic regression
3. ✅ Implemented nearest neighbor matching with caliper
4. ✅ Assessed covariate balance using SMD
5. ✅ Estimated the ATT and compared to naive estimate

**Key Insight:** The naive estimate was biased because healthier patients self-selected into treatment. PSM corrected this by comparing treated patients to similar controls.

---

## 📝 Reflection Questions

1. Why was the naive estimate biased toward zero?

2. What would happen if we had unmeasured confounders (e.g., family support)?

3. How would you explain PSM to a non-technical stakeholder?

---

## 🚀 Next Steps

- Try different caliper values — how do results change?
- Try matching WITH replacement — does ATT improve?
- Review the complete tutorial for deeper understanding

---

**Great work! You've successfully implemented Propensity Score Matching!** 🎯